# Task 1

In [15]:
%%file cs4650hw2_part1.py
from mrjob.job import MRJob

class MaxMinValue(MRJob):

    def mapper(self, _, line):
        col, row, value = line.split(",")
        value = int(value)

        # Yield column, value for max calculation in reducer
        yield ("COLUMN", col), value

        # Yield row, value for min calculation in reducer
        yield ("ROW", row), value

    def reducer(self, key, values):
        key_type, col_or_row = key

        # For columns, we want max value
        if key_type == "COLUMN":
            yield col_or_row, max(values)
        else:  # For rows, we want min value
            yield col_or_row, min(values)


if __name__ == '__main__':
    MaxMinValue.run()


Overwriting cs4650hw2_part1.py


In [16]:
!python cs4650hw2_part1.py cs4650hw2.dat

"A"	994
"B"	999
"C"	987
"D"	995
"E"	998
"F"	997
"G"	997
"H"	997
"I"	995
"J"	992
"K"	0
"L"	3
"M"	0
"N"	13
"O"	1
"P"	12
"Q"	0
"R"	2
"S"	5
"T"	2


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part1.peter.20231004.092917.512937
Running step 1 of 1...
job output is in C:\Users\peter\AppData\Local\Temp\cs4650hw2_part1.peter.20231004.092917.512937\output
Streaming final output from C:\Users\peter\AppData\Local\Temp\cs4650hw2_part1.peter.20231004.092917.512937\output...
Removing temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part1.peter.20231004.092917.512937...


# Task 2

In [17]:
%%file cs4650hw2_part2.py
from mrjob.job import MRJob


class MaxMinValueWithExample(MRJob):

    def mapper(self, _, line):
        col, row, value = line.split(',')
        value = int(value)
        # Emit column key with value and row example
        yield ("COLUMN", col), (value, row)
        # Emit row key with value and column example
        yield ("ROW", row), (value, col)

    def reducer(self, key, values):
        key_type, key_name = key
        if key_type == "COLUMN":
            # Find the maximum value for columns and associated row
            max_value, example = max(values, key=lambda x: x[0])
            yield key_name, {"value": max_value, "example": example}
        else:
            # Find the minimum value for rows and associated column
            min_value, example = min(values, key=lambda x: x[0])
            yield key_name, {"value": min_value, "example": example}


if __name__ == '__main__':
    MaxMinValueWithExample.run()

Overwriting cs4650hw2_part2.py


In [18]:
!python cs4650hw2_part2.py cs4650hw2.dat


"A"	{"value":994,"example":"R"}
"B"	{"value":999,"example":"N"}
"C"	{"value":987,"example":"S"}
"D"	{"value":995,"example":"M"}
"E"	{"value":998,"example":"N"}
"F"	{"value":997,"example":"M"}
"G"	{"value":997,"example":"M"}
"H"	{"value":997,"example":"P"}
"I"	{"value":995,"example":"S"}
"J"	{"value":992,"example":"K"}
"K"	{"value":0,"example":"H"}
"L"	{"value":3,"example":"H"}
"M"	{"value":0,"example":"E"}
"N"	{"value":13,"example":"B"}
"O"	{"value":1,"example":"H"}
"P"	{"value":12,"example":"B"}
"Q"	{"value":0,"example":"C"}
"R"	{"value":2,"example":"F"}
"S"	{"value":5,"example":"I"}
"T"	{"value":2,"example":"G"}


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part2.peter.20231004.092918.708965
Running step 1 of 1...
job output is in C:\Users\peter\AppData\Local\Temp\cs4650hw2_part2.peter.20231004.092918.708965\output
Streaming final output from C:\Users\peter\AppData\Local\Temp\cs4650hw2_part2.peter.20231004.092918.708965\output...
Removing temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part2.peter.20231004.092918.708965...


# Task 3

In [23]:
%%file cs4650hw2_part3.py

from mrjob.job import MRJob

class MaxMinValueMultipleExamples(MRJob):

    def mapper(self, _, line):
        col, row, value = line.split(',')
        value = int(value)
        # Emit column key with value and row example
        yield ("COLUMN", col), (value, row)
        # Emit row key with value and column example
        yield ("ROW", row), (value, col)

    def reducer(self, key, values):
        key_type, key_name = key
        values_list = list(values)  # Convert values to a list for multiple iterations
        examples = []

        if key_type == "COLUMN":
            # Find the maximum value for columns
            max_value = max(val for val, _ in values_list)
            examples = [example for val, example in values_list if val == max_value]
            yield key_name, {"value": max_value, "examples": examples}
        else:
            # Find the minimum value for rows
            min_value = min(val for val, _ in values_list)
            examples = [example for val, example in values_list if val == min_value]
            yield key_name, {"value": min_value, "examples": examples}

if __name__ == '__main__':
    MaxMinValueMultipleExamples.run()


Overwriting cs4650hw2_part3.py


In [24]:
!python cs4650hw2_part3.py cs4650hw2.dat


"A"	{"value":994,"examples":["R","P"]}
"B"	{"value":999,"examples":["N"]}
"C"	{"value":987,"examples":["S"]}
"D"	{"value":995,"examples":["M"]}
"E"	{"value":998,"examples":["N","S"]}
"F"	{"value":997,"examples":["M"]}
"G"	{"value":997,"examples":["M"]}
"H"	{"value":997,"examples":["P"]}
"I"	{"value":995,"examples":["S"]}
"J"	{"value":992,"examples":["K"]}
"K"	{"value":0,"examples":["H"]}
"L"	{"value":3,"examples":["H","B"]}
"M"	{"value":0,"examples":["E"]}
"N"	{"value":13,"examples":["B"]}
"O"	{"value":1,"examples":["H"]}
"P"	{"value":12,"examples":["B"]}
"Q"	{"value":0,"examples":["C","G"]}
"R"	{"value":2,"examples":["F"]}
"S"	{"value":5,"examples":["I"]}
"T"	{"value":2,"examples":["G"]}


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part3.peter.20231004.095731.366111
Running step 1 of 1...
job output is in C:\Users\peter\AppData\Local\Temp\cs4650hw2_part3.peter.20231004.095731.366111\output
Streaming final output from C:\Users\peter\AppData\Local\Temp\cs4650hw2_part3.peter.20231004.095731.366111\output...
Removing temp directory C:\Users\peter\AppData\Local\Temp\cs4650hw2_part3.peter.20231004.095731.366111...
